In [ ]:
import pandas as pd
import numpy as np
df = pd.read_csv('data.csv')
df = df.drop(['beats'], axis=1)
df.head()

#Preprocessing
df['class_name'].unique()
array(['blues', 'classical', 'country', 'disco', 'hiphop', 'jazz',
'metal', 'pop', 'reggae', 'rock'], dtype=object)
df['class_name'] = df['class_name'].astype('category')
df['class_label'] = df['class_name'].cat.codes
lookup_genre_name = dict(zip(df.class_label.unique(),
df.class_name.unique()))
lookup_genre_name

df['class_name'].unique()

cols = list(df.columns)
cols.remove('label')
cols.remove('class_label')
cols.remove('class_name')

#Data Splitting for Training and Testing
%matplotlib notebook
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
X = df.iloc[:,1:28]
y = df['class_label']
X_train, X_test, y_train, y_test = train_test_split(X, y,
random_state=3)

#Min-Max Normalization
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
X_train_scaled = scaler.fit_transform(X_train)
# we must apply the scaling to the test set that we computed for the
#training set
X_test_scaled = scaler.transform(X_test)

#Feature importance using Random Forest
from sklearn.ensemble import RandomForestClassifier
%matplotlib notebook
clf = RandomForestClassifier(random_state=0, n_jobs=-
1).fit(X_train_scaled, y_train)
importances = clf.feature_importances_
indices = np.argsort(importances)[::-1]
names = [X.columns.values[i] for i in indices]
plt.figure()
plt.title("Feature Importance")
plt.bar(range(X.shape[1]), importances[indices])
plt.xticks(range(X.shape[1]), names, rotation=90)
plt.show()

#Feature importance using Decision Tree
from sklearn.tree import DecisionTreeClassifier
clf = DecisionTreeClassifier(random_state=0).fit(X_train_scaled,
y_train)
importances = clf.feature_importances_
indices = np.argsort(importances)[::-1]
names = [X.columns.values[i] for i in indices]
plt.figure()
plt.title("Feature Importance")
plt.bar(range(X.shape[1]), importances[indices])
plt.xticks(range(X.shape[1]), names, rotation=90)
plt.show()

#K-Nearest Neighbors
from sklearn.neighbors import KNeighborsClassifier
knn = KNeighborsClassifier(n_neighbors = 13)
knn.fit(X_train_scaled, y_train)
knn.score(X_test_scaled, y_test)

#Fetching File Metadata For Prediction
from Metadata import getmetadata
a = getmetadata("3.wav")

#Predicting using KNN
d1 =np.array(a)
data1 = scaler.transform([d1])
genre_prediction = knn.predict(data1)
print(lookup_genre_name[genre_prediction[0]])

#Support Vector Machine
from sklearn.svm import SVC
clf = SVC(kernel = 'linear', C=10).fit(X_train_scaled, y_train)
clf.score(X_test_scaled, y_test)

#Predicting Using SVM
d1 =np.array(a)
data1 = scaler.transform([d1])
genre_prediction = clf.predict(data1)
print(lookup_genre_name[genre_prediction[0]])
import pickle
pick1 = {
'norma':scaler,
'svmp':knn,
'lgn':lookup_genre_name
} 
pickle.dump(pick1, open( 'models' +".p", "wb" ))